# Data preparation

##**Dataset Import from kaggle**

I have uploaded the necessary file to Kaggle as a private dataset, which should enable us to import it more quickly into Google Colab. Let's streamline the import process.

In [1]:
from google.colab import files

In [2]:
# Import the "files" module from the "google.colab" package to easily upload and download files in the Google Colaboratory environment.
from google.colab import files


In [3]:
#Here you just need to upload the API (kaggle.json) file I will be giving you.
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"mohammadjaberhossain","key":"6869aff67a258c809fca6a89975b115e"}'}

In [4]:
!pip install -q kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


In [5]:
!pip install Monai
!pip install matplotlib
!pip install numpy
!pip install tqdm
!pip install glob2
!pip install dicom2nifti
!pip install pytest-shutil
!pip install nibabel
!pip install medpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 24.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 105.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.8/151.8 kB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.7/52.7 MB 30.5 MB/s eta 0:00:00
  Created wheel for medpy: filename=MedPy-0.4.0-py3-none-any.whl size=214945 sha256=f4775212803c0d3cf56a9d7bd83ccedb47e7ec26c3c12f0f9d3581a94b036fb1
  Stored in directory: /root/.cache/pip/wheels/d4/32/c7/6380ab2edb8cca018d39a0f1d43250fd9791922c963117de46
Successfully built medpy


##**File handling together**

In [6]:
import os
os.environ["KAGGLE_CONFIG_DIR"] = "/content"

In [7]:
# Import the "os" module
import os
# Set the environment variable "KAGGLE_CONFIG_DIR" to the "/content" directory.
os.environ["KAGGLE_CONFIG_DIR"] = "/content"

In [8]:
!kaggle datasets download -d mohammadjaberhossain/finalmicedata

 99% 1.71G/1.72G [00:12<00:00, 147MB/s]
100% 1.72G/1.72G [00:12<00:00, 151MB/s]


In [9]:
#Unzipp the imported zip file
!7z x finalmicedata.zip


7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,4 CPUs Intel(R) Xeon(R) CPU @ 2.30GHz (306F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan         1 file, 1844136352 bytes (1759 MiB)

Extracting archive: finalmicedata.zip
--
Path = finalmicedata.zip
Type = zip
Physical Size = 1844136352

  0%      0% 2 - data_folder/test_image/200170928CH_Exp8_M33.nii                                                          0% 4 - data_folder/test_image/20170222CH_Exp3_zfp_M07.nii                                                             1% 6 - data_folder/test_image/20170405CH_SC_M15.nii         

In [10]:
folder_path = '/content/data_folder/train_image'
# get the list of all files in the folder
file_list = os.listdir(folder_path)
# count the number of files in the folder
num_files = len(file_list)
print("Number of files in Train Image folder:", num_files)

folder_path = '/content/data_folder/train_mask'
# get the list of all files in the folder
file_list = os.listdir(folder_path)
# count the number of files in the folder
num_files = len(file_list)
print("Number of files in train mask folder:", num_files)

folder_path = '/content/data_folder/test_image'
# get the list of all files in the folder
file_list = os.listdir(folder_path)
# count the number of files in the folder
num_files = len(file_list)
print("Number of files in test image folder:", num_files)

folder_path = test_mask_path = '/content/data_folder/test_mask'
# get the list of all files in the folder
file_list = os.listdir(folder_path)
# count the number of files in the folder
num_files = len(file_list)
print("Number of files in test Mask folder:", num_files)

folder_path = '/content/data_folder/val_image'
# get the list of all files in the folder
file_list = os.listdir(folder_path)
# count the number of files in the folder
num_files = len(file_list)
print("Number of files in val image folder:", num_files)

folder_path = '/content/data_folder/val_mask'
# get the list of all files in the folder
file_list = os.listdir(folder_path)
# count the number of files in the folder
num_files = len(file_list)
print("Number of files in val mask folder:", num_files)

Number of files in Train Image folder: 231
Number of files in train mask folder: 231
Number of files in test image folder: 66
Number of files in test Mask folder: 66
Number of files in val image folder: 34
Number of files in val mask folder: 34


## 8*2= 16 images

In [11]:
# Import necessary libraries
import os
import numpy as np
import nibabel as nib
import shutil
np.random.seed(42)

# Define the paths to the image and mask folders
image_folder = "/content/data_folder/train_image"
mask_folder = "/content/data_folder/train_mask"

# Create a list of all mask file names
mask_files = os.listdir(mask_folder)

# Initialize a dictionary to store lesion sizes
lesion_sizes = {}

# Iterate over the mask files
for mask_file in mask_files:
    if mask_file.endswith("_mask.nii"):
        # Extract the corresponding image file name
        image_file = mask_file[:-9] + ".nii"

        # Load the image file and its header
        image_path = os.path.join(image_folder, image_file)
        image_header = nib.load(image_path).header

        # Obtain the voxel size from the image header
        voxel_size = np.prod(image_header.get_zooms())

        # Load the mask file
        mask_path = os.path.join(mask_folder, mask_file)
        mask_data = nib.load(mask_path).get_fdata()

        # Calculate the size of the lesion in millimeters
        lesion_size = np.sum(mask_data) * voxel_size

        # Store the lesion size in the dictionary
        lesion_sizes[image_file] = lesion_size

# Convert the lesion sizes to an array
lesion_sizes_array = np.array(list(lesion_sizes.values()))

# Generate bins for the Gaussian distribution based on lesion sizes
bins = np.linspace(np.min(lesion_sizes_array), np.max(lesion_sizes_array), 9) # create 8 bins
class_labels = np.digitize(lesion_sizes_array, bins)

# Initialize a dictionary to store the selected images
selected_images = {}

# Select 5 images from each class based on the Gaussian distribution
for class_label in range(1, 9):
    class_indices = np.where(class_labels == class_label)[0]
    if len(class_indices) > 0:
        selected_indices = np.random.choice(class_indices, min(2, len(class_indices)), replace=False) # choose 5 or less if not available
        selected_images_from_class = [list(lesion_sizes.keys())[i] for i in selected_indices]
        selected_images.update({img: class_label for img in selected_images_from_class})

# If less than 40 images were selected, select additional images randomly
while len(selected_images) < 16:
    for class_label in range(1, 9):
        class_indices = np.where(class_labels == class_label)[0]
        leftover_indices = [idx for idx in class_indices if list(lesion_sizes.keys())[idx] not in selected_images]
        if len(leftover_indices) > 0:
            selected_index = np.random.choice(leftover_indices)
            selected_image = list(lesion_sizes.keys())[selected_index]
            selected_images[selected_image] = class_label
            if len(selected_images) >= 16:
                break

# Define the paths to the output folders
output_image_folder = "/content/data_folder/selected_images16"
output_mask_folder = "/content/data_folder/selected_masks16"

# Create the output folders if they don't exist
os.makedirs(output_image_folder, exist_ok=True)
os.makedirs(output_mask_folder, exist_ok=True)

# Copy the selected images and masks to the output folders
for selected_image in selected_images:
    # Copy the image
    source_image_path = os.path.join(image_folder, selected_image)
    destination_image_path = os.path.join(output_image_folder, selected_image)
    shutil.copy2(source_image_path, destination_image_path)

    # Copy the mask
    mask_name = selected_image[:-4] + "_mask.nii"
    source_mask_path = os.path.join(mask_folder, mask_name)
    destination_mask_path = os.path.join(output_mask_folder, mask_name)
    shutil.copy2(source_mask_path, destination_mask_path)

    # Print the information
    print("Image:", selected_image)
    print("Mask:", mask_name)
    print("Class:", selected_images[selected_image])
    print("Lesion Size (mm^3):", lesion_sizes[selected_image])
    print()

print("Selected images and masks have been copied to separate folders.")



Image: 20170921CH_Exp09_M13.nii
Mask: 20170921CH_Exp09_M13_mask.nii
Class: 1
Lesion Size (mm^3): 13.03499849466607

Image: 20180323CH_Exp3_Phag_M13.nii
Mask: 20180323CH_Exp3_Phag_M13_mask.nii
Class: 1
Lesion Size (mm^3): 8.359998255968094

Image: 20190820CH_Exp14_Senp7_M13.nii
Mask: 20190820CH_Exp14_Senp7_M13_mask.nii
Class: 2
Lesion Size (mm^3): 20.71499374927953

Image: 20190320CH_Exp4_M11.nii
Mask: 20190320CH_Exp4_M11_mask.nii
Class: 2
Lesion Size (mm^3): 20.025007012300193

Image: 20180802CH_Exp9_M25.nii
Mask: 20180802CH_Exp9_M25_mask.nii
Class: 3
Lesion Size (mm^3): 47.715021152514964

Image: 20180710CH_Exp8_M03.nii
Mask: 20180710CH_Exp8_M03_mask.nii
Class: 3
Lesion Size (mm^3): 38.66998472996056

Image: 20190220CH_Exp1_M47.nii
Mask: 20190220CH_Exp1_M47_mask.nii
Class: 4
Lesion Size (mm^3): 55.38002455048263

Image: 20180801CH_Exp9_M15.nii
Mask: 20180801CH_Exp9_M15_mask.nii
Class: 4
Lesion Size (mm^3): 67.81003006082028

Image: 20170712CH_Exp7_ZFPtm1a_M02.nii
Mask: 20170712CH_Exp7

In [12]:
folder_path = '/content/data_folder/selected_images16'
# get the list of all files in the folder
file_list = os.listdir(folder_path)
# count the number of files in the folder
num_files = len(file_list)
print("Number of files in Train Image folder:", num_files)

folder_path = '/content/data_folder/selected_masks16'
# get the list of all files in the folder
file_list = os.listdir(folder_path)
# count the number of files in the folder
num_files = len(file_list)
print("Number of files in train mask folder:", num_files)



Number of files in Train Image folder: 16
Number of files in train mask folder: 16
